In [61]:
import subprocess
import glob

In [66]:
files = glob.glob("/home/malavika/thecolumn_dems/*")
i = 0
for file in files:
    if(i%4 == 0):
        subprocess.call(["python", "formwork.py", file, file.replace(".tif","-colorised.tif"), "--k", "11", "--max_value", "103", "--min_value", "93"])
    i = i + 1 

In [7]:
import csv
with open("/home/malavika/clrTxtFile.txt") as csvfile:
    reader = csv.reader(csvfile, delimiter=' ')
    data = list(reader)
    print(len(data))

11
